In [17]:
import numpy as np
from tifffile import imread, imsave
import os
import re
import glob
import warnings
import imageio
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [18]:
def stitch_full_plane(img_dir, z_index, tile_shape):
    """
    Stitch together all the tiles of a given Z-plane for RGB images.
    """
    # Find all files for this Z-plane
    files = glob.glob(os.path.join(img_dir, f'shot__s{str(z_index).zfill(2)}_*.png'))

    # Regular expression to extract Y and X coordinates
    coord_pattern = re.compile(r'_Y(\d+)_X(\d+)\.png$')

    # Determine the dimensions of the full plane
    num_rows = max(int(coord_pattern.search(f).group(1)) for f in files) + 1
    num_cols = max(int(coord_pattern.search(f).group(2)) for f in files) + 1
    full_plane = np.zeros((num_rows * tile_shape[0], num_cols * tile_shape[1], 3), dtype=np.uint8)

    # Stitch the tiles
    for file in files:
        match = coord_pattern.search(file)
        if match:
            y, x = map(int, match.groups())
            tile = imageio.imread(file)  # Use imageio to read the image
            # Ensure that the tile is RGB
            if tile.ndim == 3 and tile.shape[2] == 3:
                full_plane[y * tile_shape[0]:(y + 1) * tile_shape[0], x * tile_shape[1]:(x + 1) * tile_shape[1], :] = tile
            else:
                print(f"Warning: Tile {file} is not an RGB image.")
        else:
            print(f"Filename does not match expected pattern: {file}")

    return full_plane

In [23]:
def crop_and_save_region(stitched_plane, bbox, output_dir, center_z, center_y, center_x, crop_mode='user_defined', crop_width=1536, crop_height=1536):
    """
    Crop a region from the stitched plane and save it, based on either a bounding box or user-defined size.
    """
    if crop_mode == 'file':
        # Crop using the bounding box from the file
        cropped_img = stitched_plane[bbox['y_min']:bbox['y_max'], bbox['x_min']:bbox['x_max']]
    elif crop_mode == 'user_defined' and crop_width is not None and crop_height is not None:
        # Crop using a user-defined size centered around (center_y, center_x)
        y_min = max(center_y - crop_height // 2, 0)
        y_max = min(center_y + crop_height // 2, stitched_plane.shape[0])
        x_min = max(center_x - crop_width // 2, 0)
        x_max = min(center_x + crop_width // 2, stitched_plane.shape[1])
        cropped_img = stitched_plane[y_min:y_max, x_min:x_max]
    else:
        raise ValueError("Invalid crop mode or missing crop dimensions for user-defined cropping.")

    output_file = os.path.join(output_dir, f'crop__z{center_z}_Y{center_y}_X{center_x}.tif')
    imsave(output_file, cropped_img)

In [20]:
def load_bounding_boxes(file_path):
    """
    Load bounding box data from the file.
    """
    bounding_boxes = []
    with open(file_path, 'r') as file:
        next(file)  # Skip header line
        for line in file:
            parts = line.strip().split(', ')
            bbox_data = {
                'object_id': int(parts[0]),
                'z_min': int(parts[1]),
                'y_min': int(parts[2]),
                'x_min': int(parts[3]),
                'z_max': int(parts[1]),
                'y_max': int(parts[4]),
                'x_max': int(parts[5]),
                'center_z' : int(parts[1]),
                'center_y': int(parts[7]),
                'center_x': int(parts[6])
            }
            bounding_boxes.append(bbox_data)
    return bounding_boxes

In [24]:
def main():
    root_dir = 'D:/shared_drive/oz'
    img_dir = os.path.join(root_dir, 'screenshot')
    bbox_file = os.path.join(root_dir, 'bounding_boxes.txt')
    output_dir = os.path.join(root_dir, 'cropped_rgb_fixed_bbox')
    tile_shape = (4096, 4096)  # Update this based on your actual tile size

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load bounding box data
    bounding_boxes = load_bounding_boxes(bbox_file)

    # Process each Z-plane
    z_indices = sorted(set(bbox['z_min'] for bbox in bounding_boxes))
    for z_index in z_indices:
        # Stitch the full plane for this Z-index
        stitched_plane = stitch_full_plane(img_dir, z_index, tile_shape)

        # Crop and save regions for this Z-plane
        for bbox in [b for b in bounding_boxes if b['z_min'] == z_index]:
            crop_and_save_region(stitched_plane, bbox, output_dir, bbox['center_z'], bbox['center_x'], bbox['center_y'])

In [25]:
if __name__ == "__main__":
    main()

MemoryError: Unable to allocate 27.0 GiB for an array with shape (98304, 98304, 3) and data type uint8